# HW4 - HAC
* Documents are represented as normalized tf-idf vectors.
* Cosine similarity for pair-wise document similarity.
* Similarity measure between clusters can be:
    * single-link, complete-link, group-average, and centroid similarity.
* To speed up your clustering … you MAY … (15 bonus points)
    * Use YOUR OWN  to obtain the cluster pair with maximal similarity.



## Import

In [4]:
import numpy as np
import pandas as pd
from collections import Counter
import pickle
import math
import glob
import os

In [5]:
from nltk.stem import PorterStemmer
p = PorterStemmer()
stemmed_doc =[]

In [6]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [7]:
from numpy import dot
from numpy.linalg import norm

## Import File

In [8]:
os.chdir('..')
os.chdir('./Data/IRTM/')
my_files = glob.glob('*.txt')

In [9]:
punctuations = '''!()-[]{};:'`"\,<>./?@#$%^&*_~'''
numbers ='1234567890'

## Functions

In [10]:
def tokenize( doc : str ) -> list:

    punctuations = '''!()-[]{};:'`"\,<>./?@#$%^&*_~'''
    numbers ='1234567890'
        
    # 處理標點符號與大小寫, 句子沒有斷乾淨
    for punc in punctuations:
        doc = doc.replace(punc, ' ').replace('\n','')
    for num in numbers:
        doc = doc.replace(num,'')
        
    doc_list = doc.lower().split(' ')
        
    stemmed_doc = [p.stem(token) for token in doc_list if p.stem(token)!= '\n' and not p.stem(token).isnumeric() and len(p.stem(token))>1 ]
        
    return stemmed_doc

In [11]:
def get_tf_and_df( my_files : list) -> dict :

    tf_dict = {} # {'idx' : {'token':tf score}}
    df_dict = {} # {'tok' : df score}

    for file in my_files:
        idx = file.strip('.txt')
        
        tf_dict[idx] = {}

        with open(file) as f:
            lines = f.read()

        stemmed_doc = tokenize(lines)

        N = len(stemmed_doc)

        # create tf_dict
        for tok in stemmed_doc:
            if tok not in tf_dict[idx]:
                tf_dict[idx][tok] = 1
            else:
                tf_dict[idx][tok] += 1
        
        # create df
        for tok in set(stemmed_doc):
            if tok not in df_dict:
                df_dict[tok] = 1
            else:
                df_dict[tok] += 1

    return tf_dict , df_dict

In [12]:
def create_tf_idf_vec( tf_dict : dict , df_dict : dict ) -> dict :

    tf_idf_dict = {} # {'idx' : {'token':tf_idf score}} 

    N = len(tf_dict)

    for idx in tf_dict:
        tf_idf_dict[idx]={}

        # normalization 
        total = sum(tf_dict[idx].values())
        tf_dict[idx] = {key: value / total for key, value in tf_dict[idx].items()}
        
        for tok in tf_dict[idx]:
            tf_idf_dict[idx][tok] = tf_dict[idx][tok] * math.log( N / df_dict[tok] )

    return tf_idf_dict

    

In [13]:
def cos_sim( idx_1 : int  ,idx_2 : int , df_dict : dict , tf_idf_dict : dict) -> int:

    idx_1 = str(idx_1)
    idx_2 = str(idx_2)

    bag_of_words = set(list(tf_idf_dict[idx_1].keys())+list(tf_idf_dict[idx_2].keys()))

    x_list = []
    y_list = []

    for word in bag_of_words:
        if word in tf_idf_dict[idx_1]:
            x_list.append(tf_idf_dict[idx_1][word])
        elif word not in  tf_idf_dict[idx_1]:
            x_list.append(0)
            
        if word in tf_idf_dict[idx_2]:
            y_list.append(tf_idf_dict[idx_2][word])
        elif word not in  tf_idf_dict[idx_2]:
            y_list.append(0)

        
    
    return dot(x_list, y_list)/(norm(x_list)*norm(y_list))

In [14]:
tf_dict , df_dict = get_tf_and_df( my_files)

In [15]:
tf_idf_dict = create_tf_idf_vec( tf_dict , df_dict )

In [16]:
print( cos_sim( 1 , 2, df_dict , tf_idf_dict) )

0.203964240589032


## HAC

In [25]:
N = len(tf_idf_dict)
C = np.array([[cos_sim(idx_1, idx_2 , df_dict , tf_idf_dict) for idx_1 in range(1, N+1)] for idx_2 in range(1, N+1)])
I = np.array([1]* N)

In [26]:
def get_max_cos( N, C, I ):

    max_sim = -1
    doc_i, doc_m = -1, -1
    for i in range(N):
        if I[i] != 1:
            continue
        for m in range(N):
            if I[m] == 1 and i != m:
                if max_sim < C[i][m]:
                    max_sim = C[i][m]
                    doc_i, doc_m = i, m
    return doc_i, doc_m
    

In [27]:
C

array([[1.        , 0.20396424, 0.30235042, ..., 0.05642569, 0.03213732,
        0.05919668],
       [0.20396424, 1.        , 0.21476783, ..., 0.04307513, 0.01393332,
        0.02685205],
       [0.30235042, 0.21476783, 1.        , ..., 0.0605471 , 0.02932991,
        0.05163034],
       ...,
       [0.05642569, 0.04307513, 0.0605471 , ..., 1.        , 0.13302009,
        0.03209313],
       [0.03213732, 0.01393332, 0.02932991, ..., 0.13302009, 1.        ,
        0.02279992],
       [0.05919668, 0.02685205, 0.05163034, ..., 0.03209313, 0.02279992,
        1.        ]])

In [28]:

for k in range(N-1):
    A = []
    i, m = get_max_cos( N, C, I )
    A.append([i ,m])
    for j in range(N):
        tmp = min(cos_sim(i+1, j+1, df_dict , tf_idf_dict), cos_sim(m+1, j+1, df_dict , tf_idf_dict))
        C[i][j] = tmp
        C[j][i] = tmp
    I[m] = 0

In [43]:
def write_result(hac_dict, cluster_num):
    with open(f"./{cluster_num}.txt", "w") as f:
        for k, v in hac_dict.items():
            for doc_id in sorted(v):
                f.write(f"{doc_id+1}\n")
            f.write("\n")

In [44]:
hac_dict = {str(i) : [i] for i in range(N)}
for doc_i, doc_m in A:
    new_element = hac_dict[str(doc_m)]
    hac_dict.pop(str(doc_m))
    hac_dict[str(doc_i)] += new_element
    if len(hac_dict) == 20:
        write_result(hac_dict, 20)
    if len(hac_dict) == 13:
        write_result(hac_dict, 13)
    if len(hac_dict) == 8:
        write_result(hac_dict, 8)

# Run All